<a href="https://colab.research.google.com/github/Aniket-76/Text-Prediction/blob/main/Copy_of_Chat_bot_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-1-public/blob/master/C3/W4/ungraded_labs/C3_W4_Lab_2_irish_lyrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [ ]:
import tensorflow as tf
import numpy as np 

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [ ]:
import os
path='/content/drive/MyDrive/Songs_Dataset_new/New romantic'
os.chdir(path)
merged_data=""
for file in os.listdir():
  if file != "English_phrases_and_sayings.txt":
    f=open(file,mode='r',encoding='utf-8')
    for line in f:
      if not line.isspace():
        line=line.replace(',',"")
        line=line.replace('\n',"")
        line=line.replace('-',"")
        line=line.replace('/',"")
        merged_data=merged_data+line
    f.close()
    merged_data+="\n"
print(merged_data)

In [ ]:
path='/content/drive/MyDrive/Songs_Dataset_new/New romantic/English_phrases_and_sayings.txt'
f=open(file,mode='r',encoding='utf-8')
for line in f:
  if not line.isspace():
    line=line.replace(',',"")
    line=line.replace('\n',"")
    line=line.replace('-',"")
    line=line.replace('/',"")
    tns=line.split(" ")
    if len(tns)>=5:
      merged_data=merged_data+line+"\n"
f.close()
merged_data+="\n"
with open("/content/sample_data/chat_bot_data.txt",mode='x',encoding='utf-8') as f:
  f.write(merged_data)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data = open('chat-bot-data.txt').read()

corpus = data.lower().split("\n")

print(corpus)

From here, you can initialize the `Tokenizer` class and generate the word index dictionary:

In [ ]:
tokenizer = Tokenizer()

tokenizer.fit_on_texts(corpus)

total_words = len(tokenizer.word_index) + 1

print(f'word index dictionary: {tokenizer.word_index}')
print(f'total words: {total_words}')

In [ ]:
input_sequences = []

for line in corpus:

	token_list = tokenizer.texts_to_sequences([line])[0]

	for i in range(1, len(token_list)):
		
		sub_phrases = token_list[:i+1]

		input_sequences.append(sub_phrases)

max_sequence_len = max([len(x) for x in input_sequences])
print(max_sequence_len)

input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

xs, labels = input_sequences[:,:-1],input_sequences[:,-1]

ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

You can then print some of the examples as a sanity check.

In [ ]:
sentence = corpus[0].split()
print(f'sample sentence: {sentence}')

token_list = []

for word in sentence: 
  token_list.append(tokenizer.word_index[word])

print(token_list)

In [ ]:
elem_number = 5

print(f'token list: {xs[elem_number]}')
print(f'decoded to text: {tokenizer.sequences_to_texts([xs[elem_number]])}')

print(f'one-hot label: {ys[elem_number]}')
print(f'index of label: {np.argmax(ys[elem_number])}')

In [ ]:
elem_number = 4

print(f'token list: {xs[elem_number]}')
print(f'decoded to text: {tokenizer.sequences_to_texts([xs[elem_number]])}')

print(f'one-hot label: {ys[elem_number]}')
print(f'index of label: {np.argmax(ys[elem_number])}')

## Build and compile the Model

Next, you will build and compile the model. We placed some of the hyperparameters at the top of the code cell so you can easily tweak it later if you want.

In [ ]:
embedding_dim = 100
lstm_units = 150
learning_rate = 0.01

model = Sequential([
          Embedding(total_words, embedding_dim, input_length=max_sequence_len-1),
          Bidirectional(LSTM(lstm_units)),
          Dense(total_words, activation='softmax')
])

model.compile(
    loss='categorical_crossentropy', 
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), 
    metrics=['accuracy']
    )

model.summary()

## Train the model

From the model summary above, you'll notice that the number of trainable params is much larger than the one in the previous lab. Consequently, that usually means a slower training time. It will take roughly 7 seconds per epoch with the GPU enabled in Colab and you'll reach around 76% accuracy after 100 epochs.

In [ ]:
epochs = 1

history = model.fit(xs, ys, epochs=epochs)

In [ ]:
model.save("AI_CHAT")

You can visualize the accuracy below to see how it fluctuates as the training progresses.

In [ ]:
!zip -r /content/AI_CHAT.zip /content/AI_CHAT

In [ ]:
from google.colab import files
files.download("/content/AI_CHAT.zip")

In [ ]:
!unzip /content/AI_CHAT1.zip -d AI_CHAT1

In [ ]:
import tensorflow as tf
from tensorflow import keras
model = tf.keras.models.load_model("AI_CHAT1/content/AI_CHAT1")

In [ ]:
embedding_layer = model.layers[0]

embedding_weights = embedding_layer.get_weights()[0]

print(embedding_weights.shape) 


In [ ]:
reverse_word_index = tokenizer.index_word

In [ ]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for word_num in range(1, total_words):

  word_name = reverse_word_index[word_num]

  word_embedding = embedding_weights[word_num]

  out_m.write(word_name + "\n")

  out_v.write('\t'.join([str(x) for x in word_embedding]) + "\n")

out_v.close()
out_m.close()


In [ ]:
# Import files utilities in Colab
try:
  from google.colab import files
except ImportError:
  pass

else:
  files.download('vecs.tsv')
  files.download('meta.tsv')


In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.show()

plot_graphs(history, 'accuracy')

## Generating Text

Now you can let the model make its own songs or poetry! Because it is trained on a much larger corpus, the results below should contain less repetitions as before. The code below picks the next word based on the highest probability output.

In [ ]:
seed_text = "जानम देख लो"

next_words = 5

for _ in range(next_words):

	token_list = tokenizer.texts_to_sequences([seed_text])[0]

	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	
	probabilities = model.predict(token_list)

	predicted = np.argmax(probabilities, axis=-1)[0]

	if predicted != 0:
		
		output_word = tokenizer.index_word[predicted]

		seed_text += " " + output_word

print(seed_text)

Here again is the code that gets the top 3 predictions and picks one at random.

## Wrap Up

This lab shows the effect of having a larger dataset to train your text generation model. As expected, this will take a longer time to prepare and train but the output will less likely become repetitive or gibberish. Try to tweak the hyperparameters and see if you get better results. You can also find some other text datasets and use it to train the model here.  